In [1]:
import pandas as pd
import fastparquet
import seaborn as sns
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [2]:
figures = pd.read_parquet('../data/figures.parquet', engine='fastparquet')
submissions = pd.read_csv('../data/submissions.csv')

In [3]:
figures.head()

,adsh,tag,reported_figure,quarterly_figure,reported_figure_py,quarterly_figure_py,computed
0,4711109000038,AccountsNotesAndLoansReceivableNetCurrent,272542000.0,272542000.0,NaN,NaN,True
1,4711109000038,AccountsPayable,279706000.0,279706000.0,NaN,NaN,False
2,4711109000038,AccountsPayableAndAccruedLiabilities,740010000.0,740010000.0,NaN,NaN,True
3,4711109000038,AccountsReceivableNetCurrent,272542000.0,272542000.0,NaN,NaN,False
4,4711109000038,AccruedIncomeTaxesPayable,0.0,0.0,NaN,NaN,False


In [4]:
submissions.head()

,adsh,cik,sic,form,period,accepted,version,amendment_adsh,is_amended,ticker,start,end,start_2,end_2,start_3,end_3,start_4,end_4
0,141057824001617,1750,3720,10-Q,2024-08-31,2024-09-23 17:45:00,2024,0,False,air,2024-06-01,2024-08-31,NaN,NaN,2023-06-01,2023-08-31,NaN,NaN
1,110465924080890,1750,3720,10-K,2024-05-31,2024-07-18 17:43:00,2023,0,False,air,2023-06-01,2024-05-31,NaN,NaN,2022-06-01,2023-05-31,NaN,NaN
2,110465924037408,1750,3720,10-Q,2024-02-29,2024-03-21 17:26:00,2023,0,False,air,2023-06-01,2024-02-29,2023-12-01,2024-02-29,2022-06-01,2023-02-28,2022-12-01,2023-02-28
3,110465923128321,1750,3720,10-Q,2023-11-30,2023-12-21 17:20:00,2023,0,False,air,2023-06-01,2023-11-30,2023-09-01,2023-11-30,2022-06-01,2022-11-30,2022-09-01,2022-11-30
4,110465923103944,1750,3720,10-Q,2023-08-31,2023-09-26 17:14:00,2023,0,False,air,2023-06-01,2023-08-31,NaN,NaN,2022-06-01,2022-08-31,NaN,NaN


In [5]:
# Merge datasets on 'adsh'
merged_data = pd.merge(figures, submissions, on='adsh')

# Filter for annual filings (10-K)
annual_filings = merged_data[merged_data['form'] == '10-K']

# Display the first few rows
annual_filings.head()

,adsh,tag,reported_figure,quarterly_figure,reported_figure_py,quarterly_figure_py,computed,cik,sic,form,...,is_amended,ticker,start,end,start_2,end_2,start_3,end_3,start_4,end_4
10895,119312509179839,AccountsNotesAndLoansReceivableNetCurrent,115802000.0,NaN,134396000.0,NaN,True,1002638,7373,10-K,...,False,otex,2008-07-01,2009-06-30,NaN,NaN,2007-07-01,2008-06-30,NaN,NaN
10896,119312509179839,AccountsPayableAndAccruedLiabilities,10356000.0,NaN,13499000.0,NaN,True,1002638,7373,10-K,...,False,otex,2008-07-01,2009-06-30,NaN,NaN,2007-07-01,2008-06-30,NaN,NaN
10897,119312509179839,AccountsPayableAndAccruedLiabilitiesNoncurrent,23073000.0,NaN,20513000.0,NaN,False,1002638,7373,10-K,...,False,otex,2008-07-01,2009-06-30,NaN,NaN,2007-07-01,2008-06-30,NaN,NaN
10898,119312509179839,AccountsReceivableNetCurrent,115802000.0,NaN,134396000.0,NaN,False,1002638,7373,10-K,...,False,otex,2008-07-01,2009-06-30,NaN,NaN,2007-07-01,2008-06-30,NaN,NaN
10899,119312509179839,AccruedIncomeTaxesPayable,10356000.0,NaN,13499000.0,NaN,False,1002638,7373,10-K,...,False,otex,2008-07-01,2009-06-30,NaN,NaN,2007-07-01,2008-06-30,NaN,NaN


In [ ]:
# List of columns to drop
columns_to_drop = [
    'start_2', 'end_2', 
    'start_3', 'end_3', 
    'start_4', 'end_4', 
    'quarterly_figure', 
    'reported_figure_py', 
    'quarterly_figure_py'
]

# Drop the specified columns
data = annual_filings.drop(columns=columns_to_drop)

In [8]:
# Ensure the 'period' column is in datetime format
data['period'] = pd.to_datetime(data['period'])

# Filter rows where the period is after 2016
# filtered_data = data[data['period'] > '2016-12-31']

In [ ]:
# Just looked if the netincome value is aligned with the actual netincome reported
data[(data["ticker"] == "aapl") &  (data["tag"] == "NetIncomeLoss")]


,adsh,tag,reported_figure,computed,cik,sic,form,period,accepted,version,amendment_adsh,is_amended,ticker,start,end
507882,119312510238044,NetIncomeLoss,1.401300e+10,False,320193,3571,10-K,2010-09-30,2010-10-27 16:36:00,2009,0,False,aapl,2009-09-27,2010-09-25
1538718,119312511282113,NetIncomeLoss,2.592200e+10,False,320193,3571,10-K,2011-09-30,2011-10-26 16:35:00,2011,0,False,aapl,2010-09-26,2011-09-24
3308421,119312512444068,NetIncomeLoss,4.173300e+10,False,320193,3571,10-K,2012-09-30,2012-10-31 17:07:00,2012,0,False,aapl,2011-09-25,2012-09-29
5130602,119312513416534,NetIncomeLoss,3.703700e+10,False,320193,3571,10-K,2013-09-30,2013-10-29 20:38:00,2013,0,False,aapl,2012-09-30,2013-09-28
6881379,119312514383437,NetIncomeLoss,3.951000e+10,False,320193,3571,10-K,2014-09-30,2014-10-27 17:12:00,2014,0,False,aapl,2013-09-29,2014-09-27
8684519,119312515356351,NetIncomeLoss,5.339400e+10,False,320193,3571,10-K,2015-09-30,2015-10-28 16:31:00,2015,0,False,aapl,2014-09-28,2015-09-26
9664678,162828016020309,NetIncomeLoss,4.568700e+10,False,320193,3571,10-K,2016-09-30,2016-10-26 16:42:00,2015,0,False,aapl,2015-09-27,2016-09-24
12156157,32019317000070,NetIncomeLoss,4.835100e+10,False,320193,3571,10-K,2017-09-30,2017-11-03 08:02:00,2017,0,False,aapl,2016-09-25,2017-09-30
14444957,32019318000145,NetIncomeLoss,5.953100e+10,False,320193,3571,10-K,2018-09-30,2018-11-05 08:02:00,2018,0,False,aapl,2017-10-01,2018-09-29
16753307,32019319000119,NetIncomeLoss,5.525600e+10,False,320193,3571,10-K,2019-09-30,2019-10-30 18:13:00,2019,0,False,aapl,2018-09-30,2019-09-28


In [18]:
data.isna().sum()

adsh               0
tag                0
reported_figure    0
computed           0
cik                0
sic                0
form               0
period             0
accepted           0
version            0
amendment_adsh     0
is_amended         0
ticker             0
start              0
end                0
dtype: int64

In [ ]:
# Number of companies in the dataset
data["ticker"].nunique()

5077

In [ ]:
# Size is pretty large (>1.5gb)
data.to_csv("../data/annual_filings.csv")